In [1]:
import pickle
import omegaconf
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

project_name = 'full_fine_tuning_50epochs_final1'

datas=( 'uc-merced-land-use-dataset', 'kvasir-dataset', 'stanford_cars', 'caltech101', 'fgvc-aircraft-2013b', 'dtd', 'flowers-102', 'oxford-iiit-pet'  ) # 'eurosat' 

losses=( 'TRADES_v2', 'CLASSIC_AT' ) #  

backbones=(
  'convnext_tiny', 'robust_convnext_tiny', 'convnext_tiny.fb_in22k', 
  'deit_small_patch16_224.fb_in1k', 'robust_deit_small_patch16_224',
  'convnext_base', 'convnext_base.fb_in22k', 'robust_convnext_base', 
  'convnext_base.clip_laion2b', 'convnext_base.clip_laion2b_augreg',
  'vit_base_patch16_224.augreg_in1k', 'vit_base_patch16_224.augreg_in21k',
  'vit_base_patch16_224.dino', 'vit_base_patch16_224.mae', 'vit_base_patch16_224.orig_in21k',
  'vit_base_patch16_224.sam_in1k', 'vit_base_patch16_224_miil.in21k', 'robust_vit_base_patch16_224' )  

import pickle
import numpy as np

final_data = []

for backbone in backbones:
    for loss in losses:
        for data in datas:

            try:

                name ='{}_{}_{}'.format(backbone, data, loss)
                with open('./results/results_{}_{}_{}_{}.pkl'.format(project_name, backbone, data, loss), 'rb') as f:
                    saved_data = pickle.load(f)
                    result = saved_data[name]["statistics"]
                    result['backbone'] = backbone
                    result['dataset'] = data
                    result['loss_function'] = loss    

                    print(name, saved_data[name]["statistics"])
            except:
                result = {}
                result['backbone'] = backbone
                result['dataset'] = data
                result['loss_function'] = loss   
                result['clean_acc'] = np.nan
                result['robust_acc'] = np.nan 
                # print('not available')

            final_data.append(result)

/home/mheuillet/Desktop/robust_training/.venv2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-18 21:37:27,792	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-02-18 21:37:28,677	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


convnext_tiny_stanford_cars_TRADES_v2 {'clean_acc': 0.7906666666666666, 'robust_acc': 0.5133333333333333, 'Linf_acc': 0.6246666666666667, 'backbone': 'convnext_tiny', 'dataset': 'stanford_cars', 'loss_function': 'TRADES_v2'}
convnext_tiny_caltech101_TRADES_v2 {'clean_acc': 0.9706666666666667, 'robust_acc': 0.906, 'backbone': 'convnext_tiny', 'dataset': 'caltech101', 'loss_function': 'TRADES_v2'}
convnext_tiny_fgvc-aircraft-2013b_TRADES_v2 {'clean_acc': 0.6653333333333333, 'robust_acc': 0.35733333333333334, 'backbone': 'convnext_tiny', 'dataset': 'fgvc-aircraft-2013b', 'loss_function': 'TRADES_v2'}
convnext_tiny_dtd_TRADES_v2 {'clean_acc': 0.7326666666666667, 'robust_acc': 0.46, 'backbone': 'convnext_tiny', 'dataset': 'dtd', 'loss_function': 'TRADES_v2'}
convnext_tiny_flowers-102_TRADES_v2 {'clean_acc': 0.9553333333333334, 'robust_acc': 0.8153333333333334, 'backbone': 'convnext_tiny', 'dataset': 'flowers-102', 'loss_function': 'TRADES_v2'}
convnext_tiny_oxford-iiit-pet_TRADES_v2 {'clean

In [2]:
import pandas as pd

df = pd.DataFrame(final_data)
df

,backbone,dataset,loss_function,clean_acc,robust_acc,Linf_acc,L2_acc,L1_acc
0,convnext_tiny,uc-merced-land-use-dataset,TRADES_v2,NaN,NaN,NaN,NaN,NaN
1,convnext_tiny,kvasir-dataset,TRADES_v2,NaN,NaN,NaN,NaN,NaN
2,convnext_tiny,stanford_cars,TRADES_v2,0.790667,0.513333,0.624667,NaN,NaN
3,convnext_tiny,caltech101,TRADES_v2,0.970667,0.906000,NaN,NaN,NaN
4,convnext_tiny,fgvc-aircraft-2013b,TRADES_v2,0.665333,0.357333,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
283,robust_vit_base_patch16_224,caltech101,CLASSIC_AT,0.193333,NaN,0.189333,NaN,NaN
284,robust_vit_base_patch16_224,fgvc-aircraft-2013b,CLASSIC_AT,0.214667,NaN,0.175333,NaN,NaN
285,robust_vit_base_patch16_224,dtd,CLASSIC_AT,0.268667,NaN,0.197333,NaN,NaN
286,robust_vit_base_patch16_224,flowers-102,CLASSIC_AT,0.006667,NaN,0.002000,NaN,NaN


In [3]:

# Define the valid condition for each row:
#   - either both are not NaN,
#   - or exactly one is NaN (and then the other must be not NaN)
valid_condition = (
    (df['robust_acc'].isna() & df['Linf_acc'].notna()) | 
    (df['Linf_acc'].isna() & df['robust_acc'].notna()) | 
    (df['robust_acc'].isna() & df['Linf_acc'].isna())
)

if valid_condition.all():
    print("Condition holds: If one column is NaN, the other is necessarily not NaN.")
else:
    print("Condition does NOT hold. The following rows do not meet the requirement:")

# set(df[~valid_condition]['dataset'])

df[~valid_condition]


Condition does NOT hold. The following rows do not meet the requirement:


,backbone,dataset,loss_function,clean_acc,robust_acc,Linf_acc,L2_acc,L1_acc
2,convnext_tiny,stanford_cars,TRADES_v2,0.790667,0.513333,0.624667,NaN,NaN
10,convnext_tiny,stanford_cars,CLASSIC_AT,0.692667,0.420000,0.422000,NaN,NaN
18,robust_convnext_tiny,stanford_cars,TRADES_v2,0.758000,0.643333,0.650000,NaN,NaN
26,robust_convnext_tiny,stanford_cars,CLASSIC_AT,0.556667,0.620000,0.414000,NaN,NaN
34,convnext_tiny.fb_in22k,stanford_cars,TRADES_v2,0.799333,0.620667,0.632667,NaN,NaN
42,convnext_tiny.fb_in22k,stanford_cars,CLASSIC_AT,0.591333,0.334000,0.346000,NaN,NaN
103,convnext_base.fb_in22k,oxford-iiit-pet,TRADES_v2,0.875333,0.733333,0.726667,NaN,NaN
263,vit_base_patch16_224_miil.in21k,oxford-iiit-pet,TRADES_v2,0.890667,0.606667,0.627333,NaN,NaN


In [4]:
df[ df['Linf_acc'].notna() ]

,backbone,dataset,loss_function,clean_acc,robust_acc,Linf_acc,L2_acc,L1_acc
2,convnext_tiny,stanford_cars,TRADES_v2,0.790667,0.513333,0.624667,NaN,NaN
7,convnext_tiny,oxford-iiit-pet,TRADES_v2,0.872667,NaN,0.690667,NaN,NaN
10,convnext_tiny,stanford_cars,CLASSIC_AT,0.692667,0.420000,0.422000,NaN,NaN
18,robust_convnext_tiny,stanford_cars,TRADES_v2,0.758000,0.643333,0.650000,NaN,NaN
23,robust_convnext_tiny,oxford-iiit-pet,TRADES_v2,0.854000,NaN,0.763333,NaN,NaN
26,robust_convnext_tiny,stanford_cars,CLASSIC_AT,0.556667,0.620000,0.414000,NaN,NaN
34,convnext_tiny.fb_in22k,stanford_cars,TRADES_v2,0.799333,0.620667,0.632667,NaN,NaN
39,convnext_tiny.fb_in22k,oxford-iiit-pet,TRADES_v2,0.838667,NaN,0.679333,NaN,NaN
42,convnext_tiny.fb_in22k,stanford_cars,CLASSIC_AT,0.591333,0.334000,0.346000,NaN,NaN
55,deit_small_patch16_224.fb_in1k,oxford-iiit-pet,TRADES_v2,0.880667,NaN,0.616667,NaN,NaN


In [5]:
df['Linf_acc'] = df['robust_acc'].combine_first(df['Linf_acc'])
# df['Linf_acc'] = df['Linf_acc'].combine_first(df['robust_acc'])

In [6]:
data_updated = df.copy()

# Combine clean_acc and robust_acc as independent metrics for ranking
data_updated['total_acc'] = data_updated['clean_acc'] + data_updated['Linf_acc']  # Sum of scores per dataset

# Aggregate total_acc across all datasets for each backbone + loss combination
aggregated_data_no_avg = data_updated.groupby(['backbone', 'loss_function']).agg(
    total_sum=('total_acc', 'sum'),  # Total sum of all scores across datasets
    total_geom_mean=('total_acc', lambda x: np.prod(x) ** (1 / len(x))),  # Geometric mean
).reset_index()

# Rank based on these combined scores
aggregated_data_no_avg['rank_total_sum'] = aggregated_data_no_avg['total_sum'].rank(ascending=False)
aggregated_data_no_avg['rank_total_geom_mean'] = aggregated_data_no_avg['total_geom_mean'].rank(ascending=False)

# Sort by sum ranking for presentation
ranked_aggregated_data_no_avg = aggregated_data_no_avg.sort_values(by='rank_total_sum')
ranked_aggregated_data_no_avg

# tools.display_dataframe_to_user(name="Ranked Backbone and Loss Combinations Without Averaging", dataframe=ranked_aggregated_data_no_avg)

,backbone,loss_function,total_sum,total_geom_mean,rank_total_sum,rank_total_geom_mean
11,convnext_tiny.fb_in22k,TRADES_v2,9.211333,1.367145,1.0,1.0
17,robust_convnext_tiny,TRADES_v2,8.998667,1.342688,2.0,2.0
31,vit_base_patch16_224.orig_in21k,TRADES_v2,8.883333,1.324775,3.0,5.0
33,vit_base_patch16_224.sam_in1k,TRADES_v2,8.826667,1.321060,4.0,6.0
9,convnext_tiny,TRADES_v2,8.730000,1.302082,5.0,8.0
25,vit_base_patch16_224.augreg_in21k,TRADES_v2,8.538000,1.268181,6.0,14.0
27,vit_base_patch16_224.dino,TRADES_v2,8.520667,1.282423,7.0,12.0
13,deit_small_patch16_224.fb_in1k,TRADES_v2,8.233333,1.248568,8.0,17.0
19,robust_deit_small_patch16_224,TRADES_v2,8.232667,1.246981,9.0,18.0
16,robust_convnext_tiny,CLASSIC_AT,8.176000,1.235778,10.0,20.0


In [7]:
# Pivot the table to group by 'backbone' and 'loss_function' in rows and create a hierarchical structure with 'dataset' as column index
grouped_df = df.pivot_table(
    index=['backbone', 'loss_function'], 
    columns='dataset', 
    values=[   'Linf_acc'  ], #'clean_acc',
    dropna=False )

grouped_df #.to_csv('results_acc.csv' )

Linf_acc            \
dataset                                         caltech101       dtd   
backbone                          loss_function                        
convnext_base                     CLASSIC_AT      0.934000  0.516667   
                                  TRADES_v2       0.940000  0.536667   
convnext_base.clip_laion2b        CLASSIC_AT      0.916000  0.469333   
                                  TRADES_v2       0.899333  0.490667   
convnext_base.clip_laion2b_augreg CLASSIC_AT      0.920667  0.495333   
                                  TRADES_v2       0.925333  0.496000   
convnext_base.fb_in22k            CLASSIC_AT      0.932000  0.531333   
                                  TRADES_v2       0.938667  0.527333   
convnext_tiny                     CLASSIC_AT      0.903333  0.390000   
                                  TRADES_v2       0.906000  0.460000   
convnext_tiny.fb_in22k            CLASSIC_AT      0.916000  0.472000   
                                  TRADES_v2       0.922000  0.502000   
deit_small_patch16_224.fb_in1k    CLASSIC_AT      0.874667  0.308667   
                                  TRADES_v2       0.880667  0.401333   
robust_convnext_base              CLASSIC_AT      0.934667  0.479333   
                                  TRADES_v2       0.948000  0.520000   
robust_convnext_tiny              CLASSIC_AT      0.930667  0.478667   
                                  TRADES_v2       0.941333  0.502000   
robust_deit_small_patch16_224     CLASSIC_AT      0.903333  0.415333   
                                  TRADES_v2       0.920667  0.430667   
robust_vit_base_patch16_224       CLASSIC_AT      0.189333  0.197333   
                                  TRADES_v2       0.927333  0.449333   
vit_base_patch16_224.augreg_in1k  CLASSIC_AT      0.858000  0.369333   
                                  TRADES_v2       0.881333  0.414000   
vit_base_patch16_224.augreg_in21k CLASSIC_AT      0.887333  0.412667   
                                  TRADES_v2       0.920667  0.388667   
vit_base_patch16_224.dino         CLASSIC_AT      0.895333  0.370000   
                                  TRADES_v2       0.886000  0.396667   
vit_base_patch16_224.mae          CLASSIC_AT      0.829333  0.320000   
                                  TRADES_v2       0.799333  0.382667   
vit_base_patch16_224.orig_in21k   CLASSIC_AT           NaN  0.376000   
                                  TRADES_v2       0.903333  0.440667   
vit_base_patch16_224.sam_in1k     CLASSIC_AT      0.832667  0.364000   
                                  TRADES_v2       0.892667  0.438000   
vit_base_patch16_224_miil.in21k   CLASSIC_AT      0.071333  0.263333   
                                  TRADES_v2       0.899333  0.398667   

                                                                     \
dataset                                         fgvc-aircraft-2013b   
backbone                          loss_function                       
convnext_base                     CLASSIC_AT               0.350667   
                                  TRADES_v2                0.627333   
convnext_base.clip_laion2b        CLASSIC_AT               0.574667   
                                  TRADES_v2                0.500667   
convnext_base.clip_laion2b_augreg CLASSIC_AT               0.582667   
                                  TRADES_v2                0.622000   
convnext_base.fb_in22k            CLASSIC_AT               0.336667   
                                  TRADES_v2                0.627333   
convnext_tiny                     CLASSIC_AT               0.392667   
                                  TRADES_v2                0.357333   
convnext_tiny.fb_in22k            CLASSIC_AT               0.048667   
                                  TRADES_v2                0.600667   
deit_small_patch16_224.fb_in1k    CLASSIC_AT               0.259333   
                                  TRADES_v2                0.456000   
robust_convnext_base              

In [26]:
import pandas as pd
import numpy as np
import plotly.express as px


df_new_cleaned = df #.drop(columns=['Unnamed: 0'])
name =  'TRADES_v2' #
df_new_cleaned = df_new_cleaned[ df_new_cleaned.loss_function != name ]

# Assuming df_new_cleaned already exists and contains all necessary columns
# Group by backbone and loss function, compute the mean and standard deviation for clean and robust accuracy
aggregated_metrics = df_new_cleaned.groupby(['backbone', 'loss_function']).agg(
    Mean_Clean_Acc=('clean_acc', 'mean'),
    Std_Clean_Acc=('clean_acc', 'std'),
    Mean_Robust_Acc=('robust_acc', 'mean'),
    Std_Robust_Acc=('robust_acc', 'std')
).reset_index()

# Classify architectures into categories
aggregated_metrics['Architecture'] = aggregated_metrics['backbone'].apply(
    lambda name: 'ConvNext' if 'convnext' in name.lower() else ('ViT/DeiT' if 'deit' in name.lower() or 'vit' in name.lower() else 'Other')
)

# Assign marker shapes based on loss function
aggregated_metrics['Marker_Shape'] = aggregated_metrics['loss_function'].apply(
    lambda loss: 'circle' if loss == 'TRADES_v2' else ('square' if loss == 'CLASSIC_AT' else 'diamond')
)

# Identify Pareto front
x = aggregated_metrics['Mean_Clean_Acc']
y = aggregated_metrics['Mean_Robust_Acc']
is_pareto = np.array([
    not any((y[j] >= y[i] and x[j] >= x[i]) for j in range(len(x)) if i != j)
    for i in range(len(x))
])
pareto_points = aggregated_metrics[is_pareto]

# Add a mapping for loss functions to valid Plotly symbols
symbol_mapping = {
    'TRADES_v2': 'circle',
    'CLASSIC_AT': 'square'
}

# Update the scatter plot with proper legend entries for loss functions
fig = px.scatter(
    aggregated_metrics,
    x='Mean_Clean_Acc',
    y='Mean_Robust_Acc',
    color='Architecture',         # Color for architecture
    symbol='loss_function',       # Symbol for loss function
    # title='Backbone Performance: Clean vs. Robust Accuracy',
    labels={
        'Mean_Clean_Acc': 'Mean Clean Accuracy',
        'Mean_Robust_Acc': 'Mean Robust Accuracy',
        'loss_function': 'Loss Function'  # Custom label for legend
    },
    color_discrete_map={
        'ConvNext': '#1f77b4',  # Blue for ConvNext
        'ViT/DeiT': '#ff7f0e',  # Orange for ViT/DeiT
        'Other': '#2ca02c'      # Green for Other architectures
    },
    symbol_map=symbol_mapping    # Apply symbol mapping
)

# Add static backbone names as labels on the figure
for i, row in aggregated_metrics.iterrows():
    fig.add_annotation(
        x=row['Mean_Clean_Acc'] - 0.01,
        y=row['Mean_Robust_Acc'] + 0.01,  # Offset text slightly above the marker
        text=row['backbone'],
        showarrow=False,
        font=dict(size=10),
        align='center'
    )

# Add Pareto front markers with larger size and same shape/color as the original
for idx, d in enumerate(pareto_points.iterrows()):
    _, row = d
    fig.add_scatter(
        x=[row['Mean_Clean_Acc']],
        y=[row['Mean_Robust_Acc']],
        mode='markers',
        showlegend=False,  # Avoid duplicating legends
        marker=dict(
            size=13,  # Larger marker size
            symbol=symbol_mapping[row['loss_function']],  # Use the mapped symbol
            color=aggregated_metrics.loc[aggregated_metrics['backbone'] == row['backbone'], 'Architecture'].map({
                'ConvNext': '#1f77b4',  # Blue for ConvNext
                'ViT/DeiT': '#ff7f0e',  # Orange for ViT/DeiT
                'Other': '#2ca02c'     # Green for Other architectures
            }).values[0]  # Map the color dynamically
        ),
        name=f"Pareto"
    )


# Add a diagonal line (x = y)
fig.add_shape(
    type="line",
    x0=0, y0=0,  # Start of line
    x1=1, y1=1,  # End of line
    xref="paper", yref="paper",  # Use normalized coordinates (0 to 1 in plot space)
    line=dict(color="red", width=2, dash="dot")  # Line style
)

# Adjust layout for better visual clarity
fig.update_layout(
    legend_title=dict(text='Backbone and Loss Function'),
    width=600,
    height=600,
    font=dict(size=12),
    #title_font=dict(size=14),
    margin=dict(l=10, r=45, t=10, b=10),
    plot_bgcolor='rgba(0,0,0,0)',
    showlegend=False,
    xaxis=dict(showgrid=True, gridcolor='lightgrey', zeroline=False),
    yaxis=dict(showgrid=True, gridcolor='lightgrey', zeroline=False),
    yaxis_range=[0,0.7],
    xaxis_range=[0,0.85]
)


fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85 ],
        ticktext=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85 ]
    ),
    yaxis=dict(
        tickmode="array",
        tickvals=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7 ],
        ticktext=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7 ]
    )
)

fig.show()
fig.write_image( "./figures/{}_{}.png".format(project_name,name) )